In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import LocalOutlierFactor
from sklearn.manifold import Isomap
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from scipy import stats

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sml-project/sample_submission.csv
/kaggle/input/sml-project/train.csv
/kaggle/input/sml-project/test.csv


# Data Analysis

In [30]:
train_df = pd.read_csv('/kaggle/input/sml-project/train.csv') #1216 samples
test_df = pd.read_csv('/kaggle/input/sml-project/test.csv') 

#Splitting into features & categories
train_x_df = train_df.drop(['ID', 'category'], axis=1)
train_y_df = train_df['category']
test_df = test_df.drop('ID', axis=1)

#Finding unique categories & count of each category [COUNTS ARE CORRECT]
category_count = {}
for i in range(len(train_y_df)):
    category = train_y_df[i]
    keys = category_count.keys()
    if (category not in keys):
        category_count[category] = 1
    else:
        category_count[category] += 1

total = 0
for key in category_count.keys():
    total += category_count[key]

#Converting category names into int labels
labels = {}
id = 0
for category in category_count.keys():
    if category not in labels.keys():
        labels[category] = id
        id += 1

#Replacing category names in train_y_df with corresponding labels
train_y_df = train_y_df.replace(to_replace = labels.keys(), value=labels.values())
print(train_y_df)

0        0
1        1
2        2
3        3
4        4
        ..
1211    14
1212     1
1213     1
1214     6
1215     6
Name: category, Length: 1216, dtype: int64


# Normalising Data

In [31]:
def normalise_data(x_train, x_test=None):
    scaler = preprocessing.StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    
    if (x_test is not None):
        x_test_scaled = scaler.transform(x_test)
    else:
        x_test_scaled = None
    
    return x_train_scaled, x_test_scaled

# Outlier Detection

In [32]:
def lof(x_train, y_train, n = 20):
    lof_ = LocalOutlierFactor(n_neighbors=n)
    predictions = lof_.fit_predict(x_train)
    
    inlier_idx = np.where(predictions==1)
    x_train = x_train[inlier_idx]
    y_train = y_train[inlier_idx]
    outlier_num = np.sum(predictions==-1)
    print(f"{outlier_num} outliers detected.")
    return x_train, y_train

# LDA

In [33]:
def lda(x_train, y_train, x_val, y_val, n, x_test=None, y_test=None):
    #performs linear discriminant analysis
    LDA = LinearDiscriminantAnalysis(solver='svd', n_components = n)
    x_train = LDA.fit_transform(x_train, y_train)
    x_val = LDA.transform(x_val)
    
    if x_test is not None and y_test is not None:
        x_test = LDA.transform(x_test)
    
    return x_train, x_val, x_test

# PCA

In [34]:
def pca(x_train, x_val, n='mle', x_test=None):
    # returns transformed data, new directions
    pca_ = PCA(n_components=n)
    x_train  = pca_.fit_transform(x_train)
    x_val = pca_.transform(x_val)
    
    if x_test is not None:
        x_test = pca_.transform(x_test)
    
    new_components = pca_.components_
    return new_components, x_train, x_val, x_test

# Models

In [35]:
def model_1():
    #Training model
    tf.keras.backend.clear_session()
    mlp_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.4, mode="columns")),
        tf.keras.layers.Dropout(rate=0.5, seed=42),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l2(0.075)),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mlp_model

    

def model_2():
    model_2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.05)),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.025)),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_2

def model_3():
    model_3 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.1, mode="columns")),
        tf.keras.layers.Dropout(rate=0.4, seed=42),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_3

def model_4():
    model_4 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.1, mode="columns")),
        tf.keras.layers.Dropout(rate=0.4, seed=42),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.025)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_4
 
def model_5():
    model_5 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.1, mode="columns")),
        tf.keras.layers.Dropout(rate=0.4, seed=42),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.025)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_5.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_5

def model_6():
    #Training model
    tf.keras.backend.clear_session()
    model_6 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.25, mode="columns")),
        tf.keras.layers.Dropout(rate=0.4, seed=9),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer =tf.keras.regularizers.l2(0.075)),
        tf.keras.layers.Dropout(rate=0.2, seed=9),
        tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.05)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_6.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_6

    

def model_7():
    model_7 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.1)),
        tf.keras.layers.Dropout(rate=0.4, seed=3),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.05)),
        tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_7.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_7

def model_8():
    model_8 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.1, mode="columns")),
        tf.keras.layers.Dropout(rate=0.3, seed=42),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.005)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_8.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_8

def model_9():
    model_9 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.2, mode="columns")),
        tf.keras.layers.Dropout(rate=0.2, seed=70),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.075)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_9.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_9

def model_10():
    model_10 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer = tf.keras.regularizers.OrthogonalRegularizer(factor=0.1, mode="columns")),
        tf.keras.layers.Dropout(rate=0.4, seed=105),
        tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(20, activation = 'softmax')
    ])
    
    model_10.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_10

# Data cleanup and pipeline definitions

In [36]:
k = 5
x_train_np = train_x_df.to_numpy() #1216x4096
y_train_np = train_y_df.to_numpy().reshape(-1,1) #1216x1
x_test_np = test_df.to_numpy()

#LOF Outlier detection and removal from training dataset
n_components_lof = 2
x_train_np, y_train_np = lof(x_train_np, y_train_np, n_components_lof)

#K-Means Clustering
kmeans_seed = 42
kmeans = KMeans(n_clusters=15, random_state = kmeans_seed)
kmeans.fit(x_train_np)
x_train_clusters = kmeans.predict(x_train_np)
x_test_clusters = kmeans.predict(x_test_np)

#Adding cluster labels as feature column
x_train_np = np.c_[x_train_np, x_train_clusters]
x_test_np = np.c_[x_test_np, x_test_clusters]

nn_1 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_1, epochs = 10)
nn_2 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_2, epochs = 10)
nn_3 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_3, epochs = 10)
nn_4 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_4, epochs = 10)
nn_5 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_5, epochs = 10)
nn_6 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_6, epochs = 10)
nn_7 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_7, epochs = 10)
nn_8 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_8, epochs = 10)
nn_9 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_9, epochs = 10)
nn_10 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = model_10, epochs = 10)

#Pipelines for the ten neural networks
pipe_1 = Pipeline([
    ('pca', PCA(n_components=150)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_1)
])

pipe_2 = Pipeline([
    ('pca', PCA(n_components=256)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_2)
])

pipe_3 = Pipeline([
    ('normalisation', preprocessing.StandardScaler()), 
    ('pca', PCA(n_components=100)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_3)
])

pipe_4 = Pipeline([
    ('quantile', QuantileTransformer(n_quantiles = 500, random_state = 2)),
    ('pca', PCA(n_components=100)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_4)
])

pipe_5 = Pipeline([
    ('pca', PCA(n_components=100)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_5)
])

pipe_6 = Pipeline([
    ('pca', PCA(n_components=200)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_6)
])

pipe_7 = Pipeline([
    ('pca', PCA(n_components=150)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_7)
])

pipe_8 = Pipeline([
    ('normalisation', preprocessing.StandardScaler()), 
    ('pca', PCA(n_components=100)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_8)
])

pipe_9 = Pipeline([
    ('normalisation', preprocessing.StandardScaler()), 
    ('pca', PCA(n_components=150)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_9)
])

pipe_10 = Pipeline([
    ('quantile', QuantileTransformer(n_quantiles = 500, random_state = 42)),
    ('pca', PCA(n_components=150)),
    ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=19)),
    ('nn', nn_10)
])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/mig

# Training the Neural Networks 

In [37]:
#NN_1
print("Training Neural Network #1")
results_1 = cross_validate(pipe_1, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_1.fit(x_train_np, y_train_np.ravel())
y_test_pred_1 = pipe_1.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_1['train_score']))
print("Testing score:", np.mean(results_1['test_score']))

Training Neural Network #1
Epoch 1/10
31/31 [==============================] - 1s 3ms/step - loss: 8.2120 - accuracy: 0.1070
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 5.8593 - accuracy: 0.3519
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 4.1049 - accuracy: 0.5556
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 2.8897 - accuracy: 0.6934
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 2.1079 - accuracy: 0.7984
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.6796 - accuracy: 0.8220
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.3389 - accuracy: 0.8488
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1814 - accuracy: 0.8498
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0377 - accuracy: 0.8704
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.7576 - accuracy:

In [38]:
#NN_2
print("Training Neural Network #2")
results_2 = cross_validate(pipe_2, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_2.fit(x_train_np, y_train_np.ravel())
y_test_pred_2 = pipe_2.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_2['train_score']))
print("Testing score:", np.mean(results_2['test_score']))

Training Neural Network #2
Epoch 1/10
31/31 [==============================] - 1s 2ms/step - loss: 6.4791 - accuracy: 0.3292
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 4.4481 - accuracy: 0.8066
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 3.2685 - accuracy: 0.9228
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 2.5902 - accuracy: 0.9537
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 2.1419 - accuracy: 0.9619
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.8283 - accuracy: 0.9640
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.6027 - accuracy: 0.9578
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.4395 - accuracy: 0.9630
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.3199 - accuracy: 0.9558
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1731 - accuracy:

In [39]:
#NN_3
print("Training Neural Network #3")
results_3 = cross_validate(pipe_3, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_3.fit(x_train_np, y_train_np.ravel())
y_test_pred_3 = pipe_3.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_3['train_score']))
print("Testing score:", np.mean(results_3['test_score']))


Training Neural Network #3
Epoch 1/10
31/31 [==============================] - 1s 2ms/step - loss: 3.5832 - accuracy: 0.1821
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 2.7118 - accuracy: 0.5329
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 2.0990 - accuracy: 0.6800
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 1.6686 - accuracy: 0.7541
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.4361 - accuracy: 0.7994
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.2804 - accuracy: 0.8179
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1666 - accuracy: 0.8230
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0920 - accuracy: 0.8272
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0361 - accuracy: 0.8374
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8010 - accuracy:

In [40]:
#NN_4
print("Training Neural Network #4")
results_4 = cross_validate(pipe_4, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_4.fit(x_train_np, y_train_np.ravel())
y_test_pred_4 = pipe_4.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_4['train_score']))
print("Testing score:", np.mean(results_4['test_score']))

Training Neural Network #4
Epoch 1/10
31/31 [==============================] - 1s 3ms/step - loss: 4.7511 - accuracy: 0.3395
Epoch 2/10
31/31 [==============================] - 0s 3ms/step - loss: 3.1885 - accuracy: 0.7119
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 2.2796 - accuracy: 0.8323
Epoch 4/10
31/31 [==============================] - 0s 3ms/step - loss: 1.7534 - accuracy: 0.8683
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.4353 - accuracy: 0.8714
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.2237 - accuracy: 0.8848
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0507 - accuracy: 0.8807
Epoch 8/10
31/31 [==============================] - 0s 3ms/step - loss: 0.9321 - accuracy: 0.8868
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8395 - accuracy: 0.8981
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.6729 - accuracy:

In [41]:
#NN_5
print("Training Neural Network #5")
results_5 = cross_validate(pipe_5, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_5.fit(x_train_np, y_train_np.ravel())
y_test_pred_5 = pipe_5.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_5['train_score']))
print("Testing score:", np.mean(results_5['test_score']))

Training Neural Network #5
Epoch 1/10
31/31 [==============================] - 1s 3ms/step - loss: 4.6502 - accuracy: 0.2377
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 3.5484 - accuracy: 0.5370
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 2.7234 - accuracy: 0.6965
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 2.1592 - accuracy: 0.7726
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.7823 - accuracy: 0.8210
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.5391 - accuracy: 0.8313
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.3520 - accuracy: 0.8560
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1983 - accuracy: 0.8591
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0953 - accuracy: 0.8539
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8304 - accuracy:

In [42]:
#NN_6
print("Training Neural Network #6")
results_6 = cross_validate(pipe_6, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_6.fit(x_train_np, y_train_np.ravel())
y_test_pred_6 = pipe_6.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_6['train_score']))
print("Testing score:", np.mean(results_6['test_score']))

Training Neural Network #6
Epoch 1/10
31/31 [==============================] - 1s 3ms/step - loss: 10.7310 - accuracy: 0.1430
Epoch 2/10
31/31 [==============================] - 0s 3ms/step - loss: 7.0068 - accuracy: 0.4619
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 4.6856 - accuracy: 0.6348
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 3.3379 - accuracy: 0.7346
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 2.5661 - accuracy: 0.7778
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 2.1175 - accuracy: 0.8251
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.8839 - accuracy: 0.8333
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.6912 - accuracy: 0.8488
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.5664 - accuracy: 0.8745
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 1.2765 - accuracy

In [43]:
#NN_7
print("Training Neural Network #7")
results_7 = cross_validate(pipe_7, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_7.fit(x_train_np, y_train_np.ravel())
y_test_pred_7 = pipe_7.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_7['train_score']))
print("Testing score:", np.mean(results_7['test_score']))

Training Neural Network #7
Epoch 1/10
31/31 [==============================] - 1s 3ms/step - loss: 11.9020 - accuracy: 0.2840
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 6.9545 - accuracy: 0.5926
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 4.2487 - accuracy: 0.7521
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 2.8754 - accuracy: 0.7932
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 2.1782 - accuracy: 0.8426
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.8193 - accuracy: 0.8508
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 1.6347 - accuracy: 0.8529
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 1.5158 - accuracy: 0.8539
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 1.4228 - accuracy: 0.8693
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 1.2501 - accuracy

In [44]:
#NN_8
print("Training Neural Network #8")
results_8 = cross_validate(pipe_8, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_8.fit(x_train_np, y_train_np.ravel())
y_test_pred_8 = pipe_8.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_8['train_score']))
print("Testing score:", np.mean(results_8['test_score']))

Training Neural Network #8
Epoch 1/10
31/31 [==============================] - 1s 2ms/step - loss: 3.0954 - accuracy: 0.2562
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 2.2287 - accuracy: 0.5957
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 1.5920 - accuracy: 0.7675
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 1.2528 - accuracy: 0.7994
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0653 - accuracy: 0.8282
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9572 - accuracy: 0.8467
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8756 - accuracy: 0.8519
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8317 - accuracy: 0.8693
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 0.7690 - accuracy: 0.8786
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.6328 - accuracy:

In [45]:
#NN_9
print("Training Neural Network #9")
results_9 = cross_validate(pipe_9, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_9.fit(x_train_np, y_train_np.ravel())
y_test_pred_9 = pipe_9.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_9['train_score']))
print("Testing score:", np.mean(results_9['test_score']))

Training Neural Network #9
Epoch 1/10
31/31 [==============================] - 1s 2ms/step - loss: 11.9953 - accuracy: 0.4465
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 6.2074 - accuracy: 0.8158
Epoch 3/10
31/31 [==============================] - 0s 3ms/step - loss: 3.2767 - accuracy: 0.8704
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 1.9265 - accuracy: 0.8837
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.3023 - accuracy: 0.8868
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0221 - accuracy: 0.8961
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 0.8799 - accuracy: 0.8920
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 0.7995 - accuracy: 0.8992
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 0.7557 - accuracy: 0.9012
Epoch 10/10
31/31 [==============================] - 0s 2ms/step - loss: 0.6793 - accuracy

In [ ]:
#NN_10
print("Training Neural Network #10")
results_10 = cross_validate(pipe_10, x_train_np, y_train_np.ravel(), cv=k, return_train_score=True)
pipe_10.fit(x_train_np, y_train_np.ravel())
y_test_pred_10 = pipe_10.predict(x_test_np).reshape(-1,1)

print("Training score:", np.mean(results_10['train_score']))
print("Testing score:", np.mean(results_10['test_score']))

Training Neural Network #10
Epoch 1/10


# Generation of final predictions by calculating modes 

In [1]:
y_test_pred = []

for i in range(y_test_pred_1.shape[0]):
    #Calculation of modes
    class_labels = np.array([y_test_pred_1[i,:], y_test_pred_2[i,:], y_test_pred_3[i,:], y_test_pred_4[i,:], y_test_pred_5[i,:], y_test_pred_6[i,:], y_test_pred_7[i,:], y_test_pred_8[i,:], y_test_pred_9[i,:], y_test_pred_10[i,:]])
    final_label = stats.mode(class_labels, axis=None)
    y_test_pred.append(final_label.mode)

#Reshaping prediction array
y_test_pred = np.array(y_test_pred).reshape(-1,1)
print(x_test_np.shape, y_test_pred.shape)

#
y_pred = []
category_names = list(labels.keys())
category_ids = list(labels.values())
for i in range(y_test_pred.shape[0]):
    cat = category_names[category_ids.index(y_test_pred[i,:])]
    y_pred.append(cat)

y_pred = np.array(y_pred).reshape(-1,1)
y_pred = np.c_[np.arange(0,y_test_pred_1.shape[0]), y_pred].reshape(-1,2)
y_pred_df = pd.DataFrame(y_pred, columns=['ID', 'Category'])
y_pred_df.to_csv('/kaggle/working/output_final_final.csv')
print("All done!")

NameError: name 'y_test_pred_1' is not defined